In [1]:
import time
import cv2
import mss
import numpy as np
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
title="number plate detection"
# ## Env setup
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils_self_created as vis_util

coordinate_list=[]
details=[]
# # Model preparation 
PATH_TO_FROZEN_GRAPH = 'frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'labelmap.pbtxt'
NUM_CLASSES = 37


# ## Load a (frozen) Tensorflow model into memory.
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


# # Detection
coordinates=[]
def image_detection(image):
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Get raw pixels from the screen, save it to a Numpy array
            image_np =cv2.imread(image)
            # To get real color we do this:
            image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            # Visualization of the results of a detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=1)
            
            
            
            # code snippet to print and license number as such.
            corresponding_coordinates=np.array(vis_util.corresponding_coordinates)
            character=np.array(vis_util.character)
            characters=[]
            for i in range(len(character)):
                for string in character[i]:
                    characters.append(string.split(':')[0])
            threshold_y=np.mean(corresponding_coordinates[:, 1])
            characters=np.array(characters)
            sortable_initial_x_coordinates=corresponding_coordinates[:, 0][corresponding_coordinates[:, 1]<threshold_y]
            starting_characters=characters[corresponding_coordinates[:, 1]<threshold_y][np.argsort(sortable_initial_x_coordinates)]
            sortable_final_x_coordinates=corresponding_coordinates[:, 0][corresponding_coordinates[:, 1]>=threshold_y]
            ending_characters=characters[corresponding_coordinates[:, 1]>=threshold_y][np.argsort(sortable_final_x_coordinates)]
            print(''.join([''.join(starting_characters), ''.join(ending_characters)]))
            
            
            
            
            # Show image with detection
            cv2.imshow(title, cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
            cv2.waitKey(0)
            cv2.destroyAllWindows()
image_detection('UP37J5654_781_393.jpg')


UP37J5654
